# Collect angular distribution symbols

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

import itertools
from typing import Iterable

import qrules
import sympy as sp
from IPython.display import Latex, Markdown

from ampform_dpd import DalitzPlotDecompositionBuilder, simplify_latex_rendering
from ampform_dpd.decay import (
    IsobarNode,
    Particle,
    ThreeBodyDecay,
    ThreeBodyDecayChain,
)
from ampform_dpd.io import as_markdown_table, aslatex, perform_cached_doit
from ampform_dpd.spin import filter_parity_violating_ls, generate_ls_couplings

simplify_latex_rendering()

In [ ]:
PDG = qrules.load_pdg()
PARTICLE_DB = {
    p.name: Particle(
        name=p.name,
        latex=p.latex,
        spin=p.spin,
        parity=int(p.parity),
        mass=p.mass,
        width=p.width,
    )
    for p in PDG
    if p.parity is not None
}
PARTICLE_DB["Lambda(2000)"] = Particle(
    name="Lambda(2000)",
    latex=R"\Lambda(2000)",
    spin=0.5,
    parity=-1,
    mass=2.0,
    width=(0.020 + 0.400) / 2,
)
Λc = PARTICLE_DB["Lambda(c)+"]
p = PARTICLE_DB["p"]
π = PARTICLE_DB["pi+"]
K = PARTICLE_DB["K-"]
PARTICLE_TO_ID = {Λc: 0, p: 1, π: 2, K: 3}
Markdown(as_markdown_table(list(PARTICLE_TO_ID)))

In [ ]:
resonance_names = [
    "Lambda(1405)",
    "Lambda(1520)",
    "Lambda(1600)",
    "Lambda(1670)",
    "Lambda(1690)",
    "Lambda(2000)",
    "Delta(1232)+",
    "Delta(1600)+",
    "Delta(1700)+",
    "K(0)*(700)+",
    "K*(892)0",
    "K(2)*(1430)0",
]
resonances = [PARTICLE_DB[name] for name in resonance_names]
Markdown(as_markdown_table(resonances))

In [ ]:
def load_three_body_decay(
    resonance_names: Iterable[str],
    particle_definitions: dict[str, Particle],
    min_ls: bool = True,
) -> ThreeBodyDecay:
    resonances = [particle_definitions[name] for name in resonance_names]
    chains: list[ThreeBodyDecayChain] = []
    for res in resonances:
        chains.extend(_create_isobar(res, min_ls))
    return ThreeBodyDecay(
        states={state_id: particle for particle, state_id in PARTICLE_TO_ID.items()},
        chains=tuple(chains),
    )


def _create_isobar(resonance: Particle, min_ls: bool) -> list[ThreeBodyDecayChain]:
    if resonance.name.startswith("K"):
        child1, child2, spectator = π, K, p
    elif resonance.name.startswith("L"):
        child1, child2, spectator = K, p, π
    elif resonance.name.startswith("D"):
        child1, child2, spectator = p, π, K
    else:
        raise NotImplementedError
    prod_ls_couplings = _generate_ls(Λc, resonance, spectator, conserve_parity=False)
    dec_ls_couplings = _generate_ls(resonance, child1, child2, conserve_parity=True)
    if min_ls:
        decay = IsobarNode(
            parent=Λc,
            child1=IsobarNode(
                parent=resonance,
                child1=child1,
                child2=child2,
                interaction=min(dec_ls_couplings),
            ),
            child2=spectator,
            interaction=min(prod_ls_couplings),
        )
        return [ThreeBodyDecayChain(decay)]
    chains = []
    for dec_ls, prod_ls in itertools.product(dec_ls_couplings, prod_ls_couplings):
        decay = IsobarNode(
            parent=Λc,
            child1=IsobarNode(
                parent=resonance,
                child1=child1,
                child2=child2,
                interaction=dec_ls,
            ),
            child2=spectator,
            interaction=prod_ls,
        )
        chains.append(ThreeBodyDecayChain(decay))
    return chains


def _generate_ls(
    parent: Particle, child1: Particle, child2: Particle, conserve_parity: bool
) -> list[tuple[int, sp.Rational]]:
    ls = generate_ls_couplings(parent.spin, child1.spin, child2.spin)
    if conserve_parity:
        return filter_parity_violating_ls(
            ls, parent.parity, child1.parity, child2.parity
        )
    return ls


DECAY = load_three_body_decay(
    resonance_names,
    particle_definitions=PARTICLE_DB,
    min_ls=True,
)
Latex(aslatex(DECAY, with_jp=True))

In [ ]:
def formulate_dynamics_symbol(
    decay_chain: ThreeBodyDecayChain,
) -> tuple[sp.Symbol, dict[sp.Symbol, float]]:
    s = sp.latex(_get_mandelstam_s(decay_chain))
    l_dec = sp.Rational(decay_chain.outgoing_ls.L)
    l_prod = sp.Rational(decay_chain.incoming_ls.L)
    s_dec = sp.Rational(decay_chain.outgoing_ls.S)
    s_prod = sp.Rational(decay_chain.incoming_ls.S)
    parameter_defaults = {}
    dynamics = sp.Symbol(
        Rf"X_{{{decay_chain.resonance.spin}}}^{{{l_prod},{s_prod};{l_dec},{s_dec}}}\left({s}\right)"
    )
    return dynamics, parameter_defaults


def _get_mandelstam_s(decay: ThreeBodyDecayChain) -> sp.Symbol:
    s1, s2, s3 = sp.symbols("sigma1:4", nonnegative=True)
    m1, m2, m3 = map(_to_mass_symbol, [p, π, K])
    decay_masses = {_to_mass_symbol(p) for p in decay.decay_products}
    if decay_masses == {m2, m3}:
        return s1
    if decay_masses == {m1, m3}:
        return s2
    if decay_masses == {m1, m2}:
        return s3
    raise NotImplementedError(
        f"Cannot find Mandelstam variable for {''.join(decay_masses)}"
    )


def _to_mass_symbol(particle: Particle) -> sp.Symbol:
    state_id = PARTICLE_TO_ID.get(particle)
    if state_id is not None:
        return sp.Symbol(f"m{state_id}", nonnegative=True)
    return sp.Symbol(f"m_{{{particle.latex}}}", nonnegative=True)

In [ ]:
model_builder = DalitzPlotDecompositionBuilder(DECAY, min_ls=(False, True))
for chain in model_builder.decay.chains:
    model_builder.dynamics_choices.register_builder(chain, formulate_dynamics_symbol)
model = model_builder.formulate(reference_subsystem=1)
model.intensity

In [ ]:
couplings_to_one = {s: 1 for s in model.parameter_defaults if "mathcal" in str(s)}

full_amplitudes = {k: perform_cached_doit(v) for k, v in model.amplitudes.items()}
full_expression = perform_cached_doit(model.intensity).xreplace(full_amplitudes)
full_expression = full_expression.doit().xreplace(couplings_to_one)
dynamic_symbols = sorted(
    (s for s in full_expression.free_symbols if str(s).startswith("X")),
    key=lambda s: (str(s)[-10:], str(s)),
)

In [ ]:
src = R"""
\begin{eqnarray}
"""
for x in dynamic_symbols:
    filter_substitutions = {s: 1 if s == x else 0 for s in dynamic_symbols}
    factor = full_expression.xreplace(filter_substitutions).simplify()
    src += Rf"{sp.latex(x)} &:& {sp.latex(factor)} \\" "\n"
src += R"\end{eqnarray}"
Latex(src)